In [ ]:
from data_generation import *

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,concat_ws, collect_list, lit,split, size, avg, udf, row_number, when
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType
#from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import max as spark_max


from datasketch import MinHash, MinHashLSH
from sklearn.cluster import KMeans
from numpy import average
import numpy as np 

import shutil
import os
import numpy as np

import time
import psutil
import matplotlib.pyplot as plt
#import pandas as pd

#os.environ["JAVA_HOME"] = "C:/Program Files/OpenJDK/jdk-21.0.2/bin"

# All functions used

In [ ]:
import random
import pandas as pd
from datetime import datetime, timedelta
import shutil
import os

def generate_case(tasks, case_id, start_time,min_datapoints,rand=True,connect = True):
    '''
    '''
    case = []
    time = start_time
    case_id=str(case_id).zfill(len(str(min_datapoints)))
    case.append(['null','S0',time,'Req',case_id]) #request between the user and the first server
    
    num_opts = len([key for key,values in tasks.items() if values[0] == 'opt'])
    #random_indexes = sorted(random.sample(list(range(len(tasks.keys()))),random.randint(2, len(tasks.keys()))))
    #list_new_tasks = [list(tasks.keys())[i] for i in sorted(random.sample(list(range(len(tasks.keys()))),random.randint(2, len(tasks.keys()))))]
    if rand == True:
        new_tasks = {task:tasks[task] for task in [list(tasks.keys())[i] for i in random.sample(list(range(len(tasks.keys())-num_opts)),random.randint(2, len(tasks.keys())-num_opts))]}
    else:
        new_tasks = tasks#{task:tasks[task] for task in [list(tasks.keys())[i] for i in sorted(random.sample(list(range(len(tasks.keys()))),random.randint(2, len(tasks.keys()))))]}
    for task,subtasks in new_tasks.items():
        if subtasks[0] != 'opt':
            server = f"S{list(tasks.keys()).index(task)+1}" #SX
            case.append(['S0',server,time,'Req',case_id])#request between the first server and the specific server
        if subtasks[0]=='one':
            service_task = random.choice(subtasks[1])
            new_server = f"{server}_{subtasks[1].index(service_task)+1}"
            #Request
            case.append([server, new_server,time,'Req',case_id])
            #Response
            time += timedelta(microseconds=random.randint(subtasks[-1][0], subtasks[-1][1])*100000)
            case.append([new_server,server,time,'Res',case_id]) #1 milisecond= 1000 microseconds
            
            
        elif subtasks[0] == 'rand':
            rand_tasks = random.sample(subtasks[1], random.randint(1, len(subtasks[1])))
            for new_task in rand_tasks:
                new_server = f"{server}_{subtasks[1].index(new_task)+1}"
                #Request
                case.append([server, new_server,time,'Req',case_id])
                #Response
                time += timedelta(microseconds=random.randint(subtasks[-1][0], subtasks[-1][1])*100000)
                case.append([new_server,server,time,'Res',case_id])


        elif subtasks[0] == 'all':
            for new_task in subtasks[1]:
                new_server = f"{server}_{subtasks[1].index(new_task)+1}"
                case.append([server, new_server,time,'Req',case_id])
                time += timedelta(microseconds=random.randint(subtasks[-1][0], subtasks[-1][1])*100000)
                case.append([new_server,server,time,'Res',case_id]) 
        
        elif subtasks[0] == 'con' and connect == True:
            service_task = random.choice(subtasks[1])
            new_server = f"{server}_{subtasks[1].index(service_task)+1}"
            #Req
            case.append([server, new_server,time,'Req',case_id])

            #Opt
            #Req
            sub_service_task =random.choice(tasks[service_task][1])
            sub_server = f"{new_server}_{list(tasks[service_task][1]).index(sub_service_task)+1}"
            case.append([new_server,sub_server,time,'Req',case_id])
            #Res
            time += timedelta(microseconds=random.randint(tasks[service_task][-1][0], tasks[service_task][-1][1])*100000)
            case.append([sub_server,new_server,time,'Res',case_id])
            #Opt end

            #Res
            case.append([new_server,server,time,'Res',case_id])
        
        elif subtasks[0] == 'opt':
            pass
        #else:
            #print('smth wrong')#need to change this to make sure it raises an error or smth like that
        
        if subtasks[0] != 'opt':
            case.append([server,'S0',time,'Res',case_id])#response between the specific server and the first server 
    case.append(['S0','null',time,'Res',case_id])#response between and the first server the user
    return case


#Random times
def random_time(start_time,end_time):
    delta = end_time - start_time
    return start_time + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

def generate_dataset(tasks, min_datapoints,start_time,end_time,random=True,connect = True,file_name = './SGD file.csv'):
    os.makedirs('data', exist_ok=True)
    datapoints = 0
    dataset = []
    user_id = 1
    while datapoints <= min_datapoints:
        time = random_time(start_time,end_time)
        case = generate_case(tasks, user_id, time,min_datapoints, random, connect= connect)
        dataset.extend(case)
        datapoints+= len(case)
        user_id+=1
    
    df = pd.DataFrame(dataset, columns=['from', 'to', 'timestamp', 'type', 'user_id'])
    output_file = "data/SDG_"+file_name+".csv"
    df.to_csv(output_file, index=False)


tasks = {'log_in': ['one',['credentials check','sing up','recover pw and log in'],(0,10)],
            'search_book': ['rand',['history','fantasy','crime','poetry','biography'],(5,15)],
            'shipment' : ['all',['adress','door number','zip code'],(15,50)],
            'payment' : ['one',['visa','master card','revolut','paypal','apple pay'],(10,100)],
            'new_site' : ['con',['site1','site2'],(10,100)],
            'site1' : ['opt',['site1_1','site1_2','site1_3'],(10,100)],
            'site2' : ['opt',['site2_1','site2_2'],(10,100)]
}   
start_time = datetime(2024, 6, 3, 9, 0, 0)  
end_time = datetime(2024, 6, 3, 10, 45, 0) 

# #Experiment 1
generate_dataset(tasks, 1000,start_time,end_time,file_name="dataset1")  
# #Experiment 2: con
# generate_dataset(tasks, 1000000,start_time,end_time,random=False,file_name="dataset1")  
# #Experiment 3
#generate_dataset(tasks, 1000000,start_time,end_time,file_name="dataset2")  


In [ ]:
################################################################################################################################
################################################# Question 1 ###################################################################
################################################################################################################################
def get_memory_usage(): 
    return resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024

def get_cpu_usage(): 
    return psutil.cpu_percent(interval=None)

def get_performance(func1,func2, vals):
    #k_values = [2, 3, 4, 5, 6, 7, 8]
    results = []

    for k in vals:
        start_time = time.time()
        start_mem = get_memory_usage()
        start_cpu = get_cpu_usage()

        replacement_candidates, minhashes = func1(df_grouped, k, 0.98)
        new_process_dictionary = func2(replacement_candidates)
        
        end_time = time.time()
        end_mem = get_memory_usage()
        end_cpu = get_cpu_usage()

        duration = end_time - start_time
        mem_used = end_mem - start_mem

        results.append({
            'k': k,
            'time_seconds': duration,
            'memory_mb': mem_used,
            'unique_processes': len(new_process_dictionary),
            'cpu': end_cpu
        })
    return results

def plot_results(results):
    k_values = [result['k'] for result in results]
    time_seconds = [result['time_seconds'] for result in results]
    cpu_percentages = [result['cpu'] for result in results]
    fig, ax1 = plt.subplots(figsize=(10, 6))

    ax1.set_xlabel('k values')
    ax1.set_ylabel('Time (seconds)', color='tab:blue')
    ax1.plot(k_values, time_seconds, marker='o', color='tab:blue', label='Time')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()
    ax2.set_ylabel('CPU Usage (%)', color='tab:red')

    ax2.plot(k_values, cpu_percentages, marker='^', color='tab:red', linestyle='--', label='CPU Usage')
    ax2.tick_params(axis='y', labelcolor='tab:red')

    plt.title('Performance Metrics for Different k Values')
    fig.legend(loc='upper left')
    plt.tight_layout()
    plt.grid(True)
    plt.show()

def plot_performances(results):
    k_values = [result['k'] for result in results]
    time_seconds = [result['time_seconds'] for result in results]
    cpu_percentages = [result['cpu'] for result in results]

    # Calculate the performance metric (Product of time_seconds and cpu)
    performance_metric = [time_seconds[i] * cpu_percentages[i] for i in range(len(results))]

    # Create a figure and axis
    plt.figure(figsize=(10, 6))

    # Plot the performance metric
    plt.plot(k_values, performance_metric, marker='o', linestyle='-', color='purple', label='Time * CPU')

    # Set labels and title
    plt.xlabel('k values')
    plt.ylabel('Performance Metric (Time * CPU)')
    plt.title('Combined Metric of Time and CPU Usage vs. k Values')
    plt.xticks(k_values)
    plt.grid(True)
    plt.legend()

    # Display the plot
    plt.tight_layout()
    plt.show()


def shingle(text, k=7):
    shingle_set = []
    for i in range(len(text)-k +1):
        shingle_set.append(text[i:i+k])
    return list(set(shingle_set))

def jaccard_similarity(list1, list2):   
    return len(set(list1).intersection(set(list2))) / len(set(list1).union(set(list2)))

def minhash_lsh(df, k_shingle, threshold):

    lsh = MinHashLSH(threshold=threshold, num_perm=128)
    minhashes = {}

    for features in df.collect():
        shingles = shingle(features["features"], k_shingle)
        m = MinHash(num_perm=128)
        for shingle_item in shingles:
            m.update(shingle_item.encode("utf8"))
        minhashes[int(features["user_id"])] = m
        lsh.insert(int(features["user_id"]), m)

    replacement_candidates = {}
    for key in lsh.keys: 
        replacement_candidates[key] = lsh.query(minhashes[key]) 

    #Key: New representative, value: Similar items
    return replacement_candidates,minhashes


def remove_dups(graph):
    keys_to_remove = []
    for key, values in graph.items():
        # Remove the key from its own value list if present
        if key in values:
            values.remove(key)
        # Mark keys for removal if their value list is empty
        if not values:
            keys_to_remove.append(key)
        else:
            graph[key] = values  # Update the graph with the cleaned value list
    
    # Remove keys that had empty value lists
    for key in keys_to_remove:
        graph.pop(key)

    return graph

def bucketing(replacement_candidates):
    #replacement_candidates = remove_dups(replacement_candidates.copy())
    visited_processes = set()
    new_process_dictionary = {}

    def bfs(start):
        queue = [start]
        bucket = []
        while queue:
            current = queue.pop(0)
            if current not in visited_processes:
                visited_processes.add(current)
                bucket.append(current)
                if current in replacement_candidates:
                    for neighbor in replacement_candidates[current]:
                        if neighbor not in visited_processes:
                            queue.append(neighbor)
        return bucket

    for key in replacement_candidates.keys():
        if key not in visited_processes:
            bucket = bfs(key)
            if bucket:
                new_process_dictionary[key] = sorted(bucket)

    return new_process_dictionary



def get_case(caseID,data):
    data1 = data.filter(data.user_id.isin([caseID]))
    data1.show()

def compare_cases(case1,case2,data):
    data1 = data.filter(data.user_id.isin([case1]))
    data2 = data.filter(data.user_id.isin([case2]))
    desired_column_list1 = data1.select("to").rdd.flatMap(lambda x: x).collect()
    desired_column_list2 = data2.select("to").rdd.flatMap(lambda x: x).collect()

    common_elements = np.intersect1d(desired_column_list1, desired_column_list2)
    union_elements = np.union1d(desired_column_list1, desired_column_list2)
    print(len(common_elements)/len(union_elements))

def get_traces(user_id,df):
    result = df.filter(col("user_id") == user_id).select("features").collect()
    if result:
        return result[0]["features"]
    else:
        return None

def get_shingles(user_id,df):
    result = df.filter(col("user_id") == user_id).select("shingles").collect()
    if result:
        return result[0]["shingles"]
    else:
        return None

def write_df(df, file_name):
    # Ensure the Output directory exists
    os.makedirs('Output', exist_ok=True)
    

    columns = df.columns
    columns.remove('user_id')
    new_column_order = columns + ['user_id']
    df = df.select(new_column_order)
    
    # Coalesce to a single partition and write the DataFrame to a text file
    df.coalesce(1).write.mode("overwrite").option("header", "true").csv('temp/temp_output')
    
    # Find the part file and move it to the desired location
    part_file = [f for f in os.listdir('temp/temp_output') if f.startswith("part-")][0]
    shutil.move(os.path.join('temp/temp_output', part_file), file_name)
    shutil.rmtree('temp/temp_output')

def output_part1(dataset, k, threshold,q=1):
    spark = SparkSession.builder.appName("OutputUserIDs").getOrCreate()
    data = spark.read.csv(dataset, header=True, inferSchema=True)
    df_filtered_m = data.filter(data.type.isin(['Req']))
    df_grouped = df_filtered_m.groupBy("user_id").agg(concat_ws("", collect_list("to")).alias("features"))
    
    replacement_candidates = minhash_lsh(df_grouped, k, threshold)
    new_process_dictionary = bucketing(replacement_candidates[0])    

    
    user_ids_to_change = []#[key for key in new_process_dictionary.keys()]#[key for key, values in new_process_dictionary.items() if len(values) > 1 or ]
    for key, values in new_process_dictionary.items():
        if len(values) >1:
            user_ids_to_change.append(key)
        elif len(values) == 1 and values[0]!= key:
            user_ids_to_change.append(key)

    user_ids_to_delete = []
    for id in user_ids_to_change:
        for case in new_process_dictionary[id]:
            if case not in user_ids_to_change:
                user_ids_to_delete.append(case)

    max_user_id = df_grouped.select(spark_max("user_id")).collect()[0][0]

    final_l = user_ids_to_change+user_ids_to_delete

    df_to_change = df_grouped.filter(col("user_id").isin(user_ids_to_change))
    distinct_user_ids_to_change = df_to_change.select("user_id").distinct()
    window_spec = Window.orderBy("user_id")
    user_id_mapping = distinct_user_ids_to_change.withColumn("new_user_id", row_number().over(window_spec) + max_user_id )
    
    df_with_new_ids = data.join(user_id_mapping, on="user_id", how="left")
    df_with_new_ids = df_with_new_ids.withColumn("user_id",
                                                 when(col("new_user_id").isNotNull(), col("new_user_id"))
                                                 .otherwise(col("user_id"))) \
                                     .drop("new_user_id")

    columns = df_with_new_ids.columns
    columns.remove('user_id')
    new_column_order = columns + ['user_id']
    df_with_new_ids = df_with_new_ids.select(new_column_order)
    
    return df_with_new_ids, user_ids_to_delete, user_ids_to_change


################################################################################################################################
################################################# Question 2 ###################################################################
################################################################################################################################

def kmeans_clustering(df, n_clusters, max_iter):
    minhashes = []
    #for jaccard verification
    minhash_dict = {}
    user_ids = []
    final_buckets = {}
    for features in df.collect():
        shingles = shingle(features["features"], 5)
        m = MinHash(num_perm=128)
        for shingle_item in shingles:
            m.update(shingle_item.encode("utf8"))
        minhashes.append(m.hashvalues)
        minhash_dict[int(features["user_id"])] = m
        user_ids.append(int(features["user_id"]))

    kmeans = KMeans(n_clusters=n_clusters, max_iter=max_iter).fit(minhashes)

    user_clusters = dict(zip(user_ids, kmeans.labels_))
    final_buckets = {}
    for key, value in user_clusters.items():
        if value in final_buckets:
            final_buckets[value].append(key)
        else:
            final_buckets[value] = [key]

    return final_buckets, minhash_dict

def get_averege_jaccard_sim(final_buckets, minhashes,get = True):
    sims = {}
    for key, value in final_buckets.items():
        for user_id_1 in final_buckets[key]:
            for user_id_2 in final_buckets[key]:
                if user_id_1 != user_id_2:
                    sig_1 = minhashes[int(user_id_1)]
                    sig_2 = minhashes[int(user_id_2)]
                    sim = MinHash.jaccard(sig_1, sig_2)
                    if key not in sims:
                        sims[key] = [sim]
                    else:
                        sims[key].append(sim)
    total_sum = 0
    total_count = 0
    sims = dict(sorted(sims.items()))
    if get == True:
        for key, value in sims.items():
            avg_sim = average(value)
            print(key, avg_sim)
            total_sum += sum(value)
            total_count += len(value)
        
        overall_average = total_sum / total_count if total_count != 0 else 0
        print("Overall Average Jaccard Similarity:", overall_average)

    return sims


# Experiments

In [ ]:
generate_dataset(tasks, 10000,start_time,end_time,file_name="dataset1")  

## Question 1

3,5,7,9

k= 7

In [ ]:
def shingle(text, k=7):
    shingle_set = []
    for i in range(len(text)-k +1):
        shingle_set.append(text[i:i+k])
    return list(set(shingle_set))

spark = SparkSession.builder.getOrCreate()

data = spark.read.csv("data/SDG_dataset2.csv", header=True, inferSchema=True)
df_filtered_m = data.filter(data.type.isin(['Req']))
df_grouped = df_filtered_m.groupBy("user_id").agg(concat_ws("",collect_list("to")).alias("features"))

shingles_udf = udf(shingle, ArrayType(StringType()))
df_shingles = df_filtered_m.groupBy("user_id").agg(concat_ws("", collect_list("to")).alias("trace")) \
    .withColumn("shingles", shingles_udf(col("trace"))) \
    .select("user_id", "shingles")

#data.show()
#df_filtered_m.show()
#df_grouped.show()
#df_shingles.show()

avg number of shingles

In [ ]:
average_num_shingles = df_shingles.withColumn("list_length", size(col("shingles"))) \
                     .agg(avg("list_length").alias("average_list_length")).collect()[0][0]
average_num_shingles
#spark.stop()

do (avg-1)/avg and use thar as threshold

In [ ]:
print(f"Initial number of cases: {df_grouped.count()}")
ans = minhash_lsh(df_grouped,7,0.97)
replacement_candidates7, minhash_dic7 = ans[0],ans[1]
new_process_dictionary7= bucketing(replacement_candidates7)
print(f"Number of unique processes after merging them with 0.97 threshold using 7-shingles: {len(new_process_dictionary7)}")

Investigate the worst case scneario, with the smallest approximate jaccard sim

In [ ]:
sims = get_averege_jaccard_sim(replacement_candidates7, minhash_dic7,get=False)

dissimilar gives you the key for the buckets with the smallest jaccard sim

In [ ]:
ans = min(set(value for key,values in sims.items() for value in values if value != 1.0))
final_values = []
for key,values in sims.items():
    for value in values:
        if value == ans:
            final_values.append(key)

dissimilar = set(final_values)

This is the minimum Jaccard similarity

In [ ]:
print(ans)

THese are the cases of which the buckets have the sim above

In [ ]:
print(dissimilar)

ANd here we get all the user_ids that have the smaller similarities and their key

In [ ]:
new_sims = []
for key in dissimilar:
    for value in replacement_candidates7[key]:
        new_sims.append((key,value,jaccard_similarity(get_shingles(value,df_shingles),get_shingles(key,df_shingles))))
investigate = [case for case in new_sims if case[-1]!=1.0]

In [ ]:
for case in investigate:
    print(f'######################### {case[0]} vs {case[1]} ################################')
    print(get_traces(case[0],df_grouped))
    print(get_traces(case[1],df_grouped))
    print('#######################################################################')

# End of what I saw with Hugo. Repeat all this for each k

### Paramter-tuning for k-shingles

First we're going to see which k is the best for the k-shingles. We gonna do such a thing by investigating how computational expensive it is to compute such minhash for different values of k

In [ ]:
results = get_performance(minhash_lsh,bucketing, [3,4,5,6,7,8,9])
plot_results(results)
plot_performances(results)

By running this a considerable amount of times we saw that the value that get a better balance between time and CPU usage is when k=7.

### Parameter-tuning for the threshold

Given that the average number of 7-shingles is 32, and we want to group processes with only small variations, we want that 31/32 shingles to be the same, so that we still allow for some small variations.

In [ ]:
print(f"Initial number of cases: {df_grouped.count()}")
ans = minhash_lsh(df_grouped,7,0.97)
replacement_candidates7, minhash_dic = ans[0],ans[1]
new_process_dictionary7= bucketing(replacement_candidates7)
print(f"Number of unique processes after merging them with 0.97 threshold using 7-shingles: {len(new_process_dictionary7)}")

After merging the processes with a threshold of 97%, using 7-shingles, we obtain 29729 candidate unique cases. In order to investigate if further analysis into the similarities needs to be done, to make sure that the false positives do not result in cases where the cases are not small variations of each other, we are going to compute the average similarities of all buckets and investigate the mininum

In [ ]:
sims = get_averege_jaccard_sim(replacement_candidates7, minhash_dic,get=False)

It's important to mention that we're still just computing the approximate jaccard similarities provided by MinHashLSH, so, in order to investigate the cases that have the smallest approximate jaccard similarities, we're going to compute the actual similarities between those cases

In [ ]:
ans = min(set(value for key,values in sims.items() for value in values if value != 1.0))
final_values = []
for key,values in sims.items():
    for value in values:
        if value == ans:
            final_values.append(key)

dissimilar = set(final_values)

In [ ]:
for case in investigate:
    print(f'######################### {case[0]} vs {case[1]} ################################')
    print(get_traces(case[0],df_grouped))
    print(get_traces(case[1],df_grouped))
    print('#######################################################################')

In [18]:
# output_part1("data/SDG_dataset2.csv",3,0.95)
# output_part1("data/SDG_dataset2.csv",5,0.95)
output_part1("data/SDG_dataset1.csv",7,0.97)

(DataFrame[from: string, to: string, timestamp: timestamp, type: string, user_id: int],
 [203,
  304,
  96,
  274,
  356,
  272,
  275,
  354,
  421,
  315,
  395,
  322,
  345,
  402,
  367,
  316,
  223,
  410,
  353,
  403,
  420,
  371,
  400],
 [14,
  23,
  32,
  68,
  114,
  117,
  132,
  146,
  147,
  154,
  181,
  185,
  201,
  240,
  246,
  276,
  294,
  346,
  386])

## Question 2

In [25]:
def get_full_trace(user, df):
    trace = []
    filtered_df = df[df['user_id'].isin(user)]
    for features in filtered_df.collect():
        trace.append(features["from"])
        trace.append(features["to"])
        trace.append(features["type"])
        trace.append(features["timestamp"])
        trace.append(features["user_id"])
    return trace

def output_part2(dataset, k, threshold,q=1):
    spark = SparkSession.builder.appName("Output2").getOrCreate()
    data = spark.read.csv(dataset, header=True, inferSchema=True)
    df_filtered_m = data.filter(data.type.isin(['Req']))
    df_grouped = df_filtered_m.groupBy("user_id").agg(concat_ws("", collect_list("to")).alias("features"))
    
    replacement_candidates, minhashes = minhash_lsh(df_grouped, k, threshold)
    new_process_dictionary = bucketing(replacement_candidates)    

    final_buckets = {}
    index = 1
    for _, value in new_process_dictionary.items():
        if len(value) > 1:
            final_buckets[index] = []
            for v in value:
                dict1 = {}
                dict1[v] = []
                dict1[v] += get_full_trace(v, data)
                final_buckets[index].append(dict1)
            index+=1
    
    #print(final_buckets)
    return new_process_dictionary,final_buckets, minhashes


new_process_dictionary,final_buckets, minhashes = output_part2("data/SDG_dataset1.csv",7,0.7)


print(final_buckets[1])


[{3: ['null', 'S0', 'Req', datetime.datetime(2024, 6, 3, 9, 10, 13), 3, 'S0', 'S2', 'Req', datetime.datetime(2024, 6, 3, 9, 10, 13), 3, 'S2', 'S2_5', 'Req', datetime.datetime(2024, 6, 3, 9, 10, 13), 3, 'S2_5', 'S2', 'Res', datetime.datetime(2024, 6, 3, 9, 10, 13, 600000), 3, 'S2', 'S0', 'Res', datetime.datetime(2024, 6, 3, 9, 10, 13, 600000), 3, 'S0', 'S3', 'Req', datetime.datetime(2024, 6, 3, 9, 10, 13, 600000), 3, 'S3', 'S3_1', 'Req', datetime.datetime(2024, 6, 3, 9, 10, 13, 600000), 3, 'S3_1', 'S3', 'Res', datetime.datetime(2024, 6, 3, 9, 10, 15, 600000), 3, 'S3', 'S3_2', 'Req', datetime.datetime(2024, 6, 3, 9, 10, 15, 600000), 3, 'S3_2', 'S3', 'Res', datetime.datetime(2024, 6, 3, 9, 10, 17, 300000), 3, 'S3', 'S3_3', 'Req', datetime.datetime(2024, 6, 3, 9, 10, 17, 300000), 3, 'S3_3', 'S3', 'Res', datetime.datetime(2024, 6, 3, 9, 10, 19, 100000), 3, 'S3', 'S0', 'Res', datetime.datetime(2024, 6, 3, 9, 10, 19, 100000), 3, 'S0', 'S1', 'Req', datetime.datetime(2024, 6, 3, 9, 10, 19, 1000

In [ ]:
groups = {}
for key, values in final_buckets.items():
    for value in values:
        print(key, value)
 

sims = get_averege_jaccard_sim(new_process_dictionary, minhashes)

### Step 1: Find and merge

In [ ]:
replacement_candidates = minhash_lsh(df_grouped,5,0.7)
new_process_dictionary = bucketing(replacement_candidates)
print(len(replacement_candidates))
print(len(new_process_dictionary))

### STEP 2: Find/cluster similar items

In [ ]:
#from sklearn.model_selection import train_test_split, GridSearchCV

users = []
for key in new_process_dictionary:
    users.append(key)

filtered_df = df_grouped[df_grouped['user_id'].isin(users)]

final_buckets, minhashes = kmeans_clustering(filtered_df,500,100)

### Verification

In [ ]:
get_averege_jaccard_sim(final_buckets, minhashes)


# Old

In [ ]:
#print(f"Initial number of cases: {df_grouped.count()}")
# replacement_candidates3 = minhash_lsh(df_grouped,3,0.98)
# new_process_dictionary3 = bucketing(replacement_candidates3)
#print(f"After merging cases with threshold 3-shingles: {len(new_process_dictionary3)}")

In [ ]:
# df_split = df_grouped.withColumn("feature_length", size(split(col("features"), "S")))
# average_length = df_split.agg(avg("feature_length")).collect()[0][0]

# windowSpec = Window.partitionBy(F.lit(1)).orderBy("feature_length")
# df_split = df_split.withColumn("row_number", F.row_number().over(windowSpec))
# total_count = df_split.count()

# if total_count % 2 == 0:
#     median_index1 = total_count // 2
#     median_index2 = median_index1 + 1
#     median_value = df_split.filter(col("row_number").isin([median_index1, median_index2])) \
#                                   .agg(avg("feature_length")).collect()[0][0]
# else:
#     median_index = (total_count // 2) + 1
#     median_value = df_split.filter(col("row_number") == median_index) \
#                                   .select("feature_length").collect()[0][0]
    
# print(f"Average number of requests: {average_length}")
# print(f"Median number of requests: {median_value}")


#Given that both the average and the median are close to 13, which shows that the dataset is symetricly distributed when it comes to how many 
#requests are performed, we gonna assume that two cases have a small variation iif the number of different requests is around 1. To get an 
# #approximation of the threshold, we're going to use the resutls shown before, so we assume that 12/13 are the same. Given this, we decided 
# to use a threshold of 95%

In [ ]:

def remove_node_from_neighbors(graph, node_to_remove):
    if node_to_remove in graph:
        # Remove node_to_remove from all adjacency lists in the graph
        for node, neighbors in graph.items():
            if node != node_to_remove:  # Skip the node_to_remove itself
                if node_to_remove in neighbors:
                    neighbors.remove(node_to_remove)

def dfs(graph, start, visited):
    stack = [start]
    result = []
    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            result.append(node)
            stack.extend(graph[node])
    return result

def transform_graph(graph):
    reachable_dict = {}
    for node in graph:
        visited = set()
        reachable_nodes = dfs(graph, node, visited)
        reachable_dict[node] = [n for n in reachable_nodes if n != node]  # Exclude the start node itself
    
    # Filter out keys that also appear as values
    keys_to_remove = set()
    for node, neighbors in graph.items():
        keys_to_remove.update(neighbors)
    
    final_dict = {k: v for k, v in reachable_dict.items() if k not in keys_to_remove}
    return final_dict

def bucketing(old_replacement_candidates):
    alt_old_replacement_candidates = old_replacement_candidates.copy()
    replacement_candidates = alt_old_replacement_candidates
    print('no dups')
    print(replacement_candidates)
    visited = set()
    for node in replacement_candidates.keys():
        if node not in visited:
            remove_node_from_neighbors(replacement_candidates, node)
        for neighbour in replacement_candidates[node]:
            visited.add(neighbour)
    # print('no neighbours')
    # print(replacement_candidates)
    new_process_dictionary = transform_graph(replacement_candidates)
    print('final')
    print(new_process_dictionary)
    
        
    return new_process_dictionary

#generate_dataset(tasks, 100000,start_time,end_time,file_name="dataset1",random=False,connect=False)  
#print('minhash')
ans = output_part1("data/SDG_dataset1.csv", 5, 0.97)
#print(ans[39])
print(ans)
#ans2 = bucketing(ans)